In [34]:
import pandas as pd
import sys
import os
import numpy
import re
import json
import pandas as pd
import re
import textacy

In [2]:
comm_file = 'CommercialTimber_all_processed.csv'
cites_file = 'CITES_list.csv'
redlist_file = 'RedList_collated.csv'
OP_FILE = 'Combined_Species_Data_v1-0.csv' 

In [3]:
main_df = pd.read_csv(
    comm_file,
    usecols= ['entity_id','sc_name']
)
main_df['family'] = None
main_df['range_state'] = None

In [4]:
main_df.columns

Index(['entity_id', 'sc_name', 'family', 'range_state'], dtype='object')

In [5]:
CITES_DF = pd.read_csv(
    cites_file,
    usecols=  ['Family','Scientific Name','Listing','All_DistributionISOCodes'],
    header=1
)
CITES_DF.columns

Index(['Family', 'Scientific Name', 'Listing', 'All_DistributionISOCodes'], dtype='object')

In [6]:
def clean_sc_name(nm):
    return re.sub('".*"', '', nm )

# Add in *flags*
### FLAGS : CITES, IUCN

In [7]:
cur_entity_id = max(list(main_df['entity_id']))+1

'''
Add in column : 'CITES','IUCN'
'''

main_df['CITES'] = 0
main_df['IUCN']  = 0

'''
Try to find if row already exists
'''
def find_in_main( df, sc_name ):
    res = df.loc[df['sc_name'] == sc_name]
    if len(res) > 0 :
        id = list(res['entity_id'])[0]
        return id
    return None

In [8]:
'''
Go through each row of CITES Data
'''
_FLAG1 = 'CITES'
_FLAG2 = 'IUCN'

for i, row in CITES_DF.iterrows():
    scn = row['Scientific Name']
    scn =  clean_sc_name(scn)
    id = find_in_main(main_df, scn)
    cites_listing = row['Listing']
    if cites_listing in ['I','II','III'] :
        flag = 1
    else :
        flag = 0
    range_state = row['All_DistributionISOCodes']
    _family = row['Family']
    
    # species present in main_df
    if id is not None :
        index = main_df.index[main_df['entity_id']==id][0]
        main_df.loc[i,_FLAG1] = flag
        main_df.loc[i,'family'] = _family
    else :
        tmp_dict = {
            'entity_id' : cur_entity_id,
            'sc_name' : scn,
            'family': _family,
             _FLAG1 : flag,
             _FLAG2 : 0,
            'range_state' : range_state
        }
        main_df = main_df.append(tmp_dict,ignore_index=True)
        cur_entity_id += 1


In [10]:
l = len(main_df[main_df['CITES']==1])
print('Number of entries with CITES == 1 ::', l)
print('Total entries ', len(main_df) )

Number of entries with CITES == 1 :: 15191
Total entries  17079


In [11]:
old_main_df = pd.DataFrame(main_df,copy=True)

In [17]:
main_df = pd.DataFrame(old_main_df,copy=True)

In [18]:
REDLIST_DF = pd.read_csv(redlist_file,index_col=0)

In [19]:
REDLIST_DF.columns

Index(['id', 'genus', 'species', 'sub_species', 'main_common_name',
       'other_common_names', 'status'],
      dtype='object')

In [20]:
main_df['common_name'] = None
status_0 = ['LR/nt', 'LC', 'LR/lc']

In [21]:
def find_in_main( df, scn):
    tmp_df = df.loc[(main_df['sc_name'] == scn )]
    
    if len(tmp_df) > 0: 
        tmp_df.reset_index(inplace = True)
        entity_id = tmp_df.loc[0,'entity_id']
        index = df.index[df['entity_id']==entity_id][0]
        return  index 
    return None 

In [22]:
_FLAG1 = 'IUCN'
_FLAG2 = 'CITES'

'''
Go through each row of IUCN Redlist Data
'''
cur_entity_id = max(list(main_df['entity_id']))+1
for i , row in REDLIST_DF.iterrows():
    gn = row['genus']
    sp = row['species']
    ssp = row['sub_species']
    res =  None
    cand_sc_names = []
    
    if type(ssp) == str:
        '''
        Try with 2 variations
        1. genus sp sub 
        2. genus sp
        '''
        sc_nm_i1 = ' '.join([gn ,sp, ssp])
        sc_nm_i2 = ' '.join([gn ,sp])
        cand_sc_names.append(sc_nm_i1)
        cand_sc_names.append(sc_nm_i2)
    else:
        if ';' in sp:
            sp_parts = sp.split(';')
        else :
            sp_parts = [sp]
        for s in sp_parts:
            sc_nm_i = ' '.join([gn ,s])
            cand_sc_names.append(sc_nm_i)
    
    status = row['status']
    
    if status in status_0 :
        flag = 0
    else :
        flag = 1
    # Common Names
    cn = []
    mcn = row['main_common_name']
    if type(mcn) == str:
        cn.append(mcn)
    
    oth_cn = row['other_common_names']
    if type(oth_cn) == str:
        oth_cn = oth_cn.split(';')
        cn.extend(oth_cn)
        
    cn = list(set(cn))
    '''
    ; separated common names
    ''' 
    COMMON_NAMES = ';'.join(cn)
    
    '''
    Check or each of the candidate scientific names
    '''
    for SN in cand_sc_names :
        search_idx = find_in_main(main_df, SN) 
        if search_idx is not None :
            # Pre-existing record
            main_df.loc[ search_idx, _FLAG1 ] = flag
            main_df.loc[ search_idx, 'common_name' ] = COMMON_NAMES
        else :
            # New record
            tmp_dict = {
                'entity_id' : cur_entity_id,
                'family': None,
                'sc_name' : SN,
                'common_name': COMMON_NAMES,
                _FLAG1 : flag,
                _FLAG2 : 0,
                'range_state': None
            }
            
            main_df = main_df.append(tmp_dict,ignore_index = True)
            cur_entity_id += 1
    


In [23]:

main_df.to_csv(OP_FILE,index=None)
print(len(main_df))

39732


In [24]:
df = pd.read_csv(OP_FILE ,index_col = 0)
df.head(10)

,sc_name,family,range_state,CITES,IUCN,common_name
entity_id,,,,,,
1,Acacia auriculiformis,NaN,NaN,0,0,Ear-leaf Acacia
2,Acacia crassicarpa,NaN,NaN,0,1,NaN
3,Acacia decurrens,NaN,NaN,0,0,NaN
4,Acacia koa,NaN,NaN,0,0,Koa;Gray Koa
5,Acacia mangium,NaN,NaN,0,0,NaN
6,Acacia mearnsii,NaN,NaN,0,0,NaN
7,Acacia melanoxylon,NaN,NaN,0,0,NaN
8,Acacia nilotica,NaN,NaN,0,0,Sunt;Qarad
9,Acacia pubescens,NaN,NaN,0,0,NaN


In [25]:
include_exclude_file = 'Inclusion_Exclusion_of_TaxonomicFamilies_forMasterSpeciesList_8July.xlsx' 
ie_df = pd.read_excel(include_exclude_file)

cols = list(ie_df.columns)
req_col = [cols[1],cols[4]]

ie_df = ie_df[req_col]
ie_df = ie_df.rename(columns={cols[1]:'family', cols[4]:'include'})
exc_df = ie_df.loc[(ie_df['include']==0)]

'''
The list of Families that are to be excluded
'''
EXCLUDE_LIST = list(exc_df['family'])
print(EXCLUDE_LIST)                    

['Agavaceae', 'Amaryllidaceae', 'Bromeliaceae', 'Cactaceae', 'Cucurbitaceae', 'Cyatheaceae', 'Cycadaceae', 'Dicksoniaceae', 'Didiereaceae', 'Dioscoreaceae', 'Droseraceae', 'Fouquieriaceae', 'Gnetaceae', 'Liliaceae', 'Nepenthaceae', 'Orchidaceae', 'Orobanchaceae', 'Papaveraceae', 'Passifloraceae', 'Pedaliaceae', 'Portulacaceae', 'Ranunculaceae', 'Sarraceniaceae', 'Scrophulariaceae', 'Stangeriaceae', 'Valerianaceae', 'Vitaceae', 'Welwitschiaceae', 'Zamiaceae', 'Zingiberaceae']


In [28]:
# Read in the family_genus json
FAMILY_GENUS_JSON_FILE = 'family_genus.json'

with open(FAMILY_GENUS_JSON_FILE,'r') as f :
    data = f.read()

FG_dict = json.loads(data)
Genus_Family_dict = {}

#create a map from genus to family
for f, g_list in FG_dict.items():
    for g in g_list :
        if g in Genus_Family_dict.keys():
            pass
        else :
            Genus_Family_dict[g] = f

### Set up the genus in each row of dataframe

In [29]:
'''
Set up the genus in each row of dataframe
'''
def set_genus(row):
    gn = (row['sc_name'].split(' '))[0]
    return gn

df['genus'] = df.apply(set_genus,axis=1)
df.head(10)

,sc_name,family,range_state,CITES,IUCN,common_name,genus
entity_id,,,,,,,
1,Acacia auriculiformis,NaN,NaN,0,0,Ear-leaf Acacia,Acacia
2,Acacia crassicarpa,NaN,NaN,0,1,NaN,Acacia
3,Acacia decurrens,NaN,NaN,0,0,NaN,Acacia
4,Acacia koa,NaN,NaN,0,0,Koa;Gray Koa,Acacia
5,Acacia mangium,NaN,NaN,0,0,NaN,Acacia
6,Acacia mearnsii,NaN,NaN,0,0,NaN,Acacia
7,Acacia melanoxylon,NaN,NaN,0,0,NaN,Acacia
8,Acacia nilotica,NaN,NaN,0,0,Sunt;Qarad,Acacia
9,Acacia pubescens,NaN,NaN,0,0,NaN,Acacia


In [31]:
def set_family(row):
    global Genus_Family_dict
    gn = row['genus']
    if type(row['family']) == str :
        return row['family']
    elif gn in Genus_Family_dict.keys():
        family = Genus_Family_dict[gn]
        return family
    return None

df['family'] = df.apply(set_family,axis=1)
df.head(10)

,sc_name,family,range_state,CITES,IUCN,common_name,genus
entity_id,,,,,,,
1,Acacia auriculiformis,Leguminosae,NaN,0,0,Ear-leaf Acacia,Acacia
2,Acacia crassicarpa,Leguminosae,NaN,0,1,NaN,Acacia
3,Acacia decurrens,Leguminosae,NaN,0,0,NaN,Acacia
4,Acacia koa,Leguminosae,NaN,0,0,Koa;Gray Koa,Acacia
5,Acacia mangium,Leguminosae,NaN,0,0,NaN,Acacia
6,Acacia mearnsii,Leguminosae,NaN,0,0,NaN,Acacia
7,Acacia melanoxylon,Leguminosae,NaN,0,0,NaN,Acacia
8,Acacia nilotica,Leguminosae,NaN,0,0,Sunt;Qarad,Acacia
9,Acacia pubescens,Leguminosae,NaN,0,0,NaN,Acacia


In [33]:
'''
Remove the records where Families are to be excluded
'''
for _exc in EXCLUDE_LIST:
    df = df.loc[(df['family']!=_exc)]

df.to_csv(OP_FILE)
print (len(df))

24685
